# SPR 2026 - BERTimbau (BERT Português)

Fine-tuning do BERTimbau para classificação BI-RADS.

**Modelo:** neuralmind/bert-base-portuguese-cased

**Formato:** Code Competition (Kaggle) / Google Colab

In [ ]:
# ============================================================
# SETUP
# ============================================================
import os
import sys

IS_KAGGLE = os.path.exists('/kaggle/input')
IS_COLAB = 'google.colab' in sys.modules

print(f"Ambiente: {'Kaggle' if IS_KAGGLE else 'Colab' if IS_COLAB else 'Local'}")

if IS_KAGGLE:
    DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
    OUTPUT_DIR = '/kaggle/working'
elif IS_COLAB:
    !pip install kaggle transformers accelerate -q
    from google.colab import userdata
    os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
    os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')
    !mkdir -p ~/.kaggle
    !echo '{"username":"'$KAGGLE_USERNAME'","key":"'$KAGGLE_KEY'"}' > ~/.kaggle/kaggle.json
    !chmod 600 ~/.kaggle/kaggle.json
    !kaggle competitions download -c spr-2026-mammography-report-classification -q
    !mkdir -p data && unzip -o -q spr-2026-mammography-report-classification.zip -d data/
    DATA_DIR = 'data'
    OUTPUT_DIR = '.'
else:
    DATA_DIR = '../data'
    OUTPUT_DIR = '.'

print(f"DATA_DIR: {DATA_DIR}")

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import warnings
warnings.filterwarnings('ignore')

# Config
SEED = 42
MODEL_NAME = 'neuralmind/bert-base-portuguese-cased'
MAX_LENGTH = 512
BATCH_SIZE = 8
EPOCHS = 3
LR = 2e-5
N_FOLDS = 5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(SEED)

## 1. Carregar Dados

In [ ]:
train = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
print(f"Train: {train.shape}")

test_path = os.path.join(DATA_DIR, 'test.csv')
if os.path.exists(test_path):
    test = pd.read_csv(test_path)
    print(f"Test: {test.shape}")
    assert set(['ID', 'report']).issubset(test.columns)
else:
    test = None
    print("test.csv não disponível - será carregado no runtime Kaggle")

## 2. Dataset

In [ ]:
class MammographyDataset(Dataset):
    def __init__(self, texts, labels=None, tokenizer=None, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        encoding = self.tokenizer(
            str(self.texts[idx]),
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt',
        )
        item = {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
        }
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

## 3. Treinar com K-Fold

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'f1_macro': f1_score(labels, predictions, average='macro')}

skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
oof_preds = np.zeros((len(train), 7))
fold_scores = []

for fold, (train_idx, val_idx) in enumerate(skf.split(train, train['target'])):
    print(f"\n{'='*50}\nFold {fold+1}/{N_FOLDS}\n{'='*50}")
    
    train_ds = MammographyDataset(
        train.iloc[train_idx]['report'].tolist(),
        train.iloc[train_idx]['target'].tolist(),
        tokenizer, MAX_LENGTH
    )
    val_ds = MammographyDataset(
        train.iloc[val_idx]['report'].tolist(),
        train.iloc[val_idx]['target'].tolist(),
        tokenizer, MAX_LENGTH
    )
    
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=7)
    
    args = TrainingArguments(
        output_dir=f'{OUTPUT_DIR}/fold_{fold}',
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE*2,
        learning_rate=LR,
        weight_decay=0.01,
        warmup_ratio=0.1,
        fp16=torch.cuda.is_available(),
        evaluation_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True,
        metric_for_best_model='f1_macro',
        greater_is_better=True,
        report_to='none',
        seed=SEED,
    )
    
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        compute_metrics=compute_metrics,
    )
    
    trainer.train()
    
    # OOF predictions
    preds = trainer.predict(val_ds)
    oof_preds[val_idx] = preds.predictions
    fold_f1 = f1_score(train.iloc[val_idx]['target'], np.argmax(preds.predictions, axis=1), average='macro')
    fold_scores.append(fold_f1)
    print(f"Fold {fold+1} F1: {fold_f1:.4f}")

print(f"\nOOF F1-Macro: {np.mean(fold_scores):.4f} (+/- {np.std(fold_scores):.4f})")

## 4. Gerar Submissão

In [ ]:
if test is not None:
    test_ds = MammographyDataset(test['report'].tolist(), None, tokenizer, MAX_LENGTH)
    test_preds = trainer.predict(test_ds)
    predictions = np.argmax(test_preds.predictions, axis=1)
    
    submission = pd.DataFrame({'ID': test['ID'], 'target': predictions})
    submission.to_csv(os.path.join(OUTPUT_DIR, 'submission.csv'), index=False)
    print(f"Submissão salva!")
    print(submission['target'].value_counts().sort_index())
else:
    print("Test não disponível - submeta este notebook no Kaggle")

In [ ]:
if IS_COLAB and test is not None:
    from google.colab import files
    files.download('submission.csv')